In [5]:
import tensorflow as tf
print(tf.__version__)
tf.enable_eager_execution()

1.14.0


In [6]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset imdb_reviews downloaded and prepared to C:\Users\mohit\tensorflow_datasets\imdb_reviews\plain_text\0.1.0. Subsequent calls will reuse this data.


In [12]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()
for s,l in train_data:
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())
  
for s,l in test_data:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())
  
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)


In [13]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [17]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"

tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen = max_length, truncating = trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen = max_length, truncating = trunc_type)


In [20]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[1]))
print(training_sentences[1])

b i absolutely loved this movie when i was a kid i cried every time i watched it it wasn't weird to me i totally identified with the characters i would love to see it again and hope i wont be disappointed <OOV> rocks i was really drawn in to the fantasy world and to me the movie was <OOV> i wonder if i ever saw the series and have confused them the acting i thought was strong i loved jack <OOV> he was so dreamy to an 10 year old when i first saw the movie not in 1970 i can still remember the characters vividly the <OOV> was totally believable and i can still <OOV> the evil woods
b"I absolutely LOVED this movie when I was a kid. I cried every time I watched it. It wasn't weird to me. I totally identified with the characters. I would love to see it again (and hope I wont be disappointed!). Pufnstuf rocks!!!! I was really drawn in to the fantasy world. And to me the movie was loooong. I wonder if I ever saw the series and have confused them? The acting I thought was strong. I loved Jack W

In [19]:
print(padded[1])
print(training_sentences[1])

[  59   11  439  455   12   18   55   11   14    4  547   11 3811  175
   60   11  298   10   10  559  919    6   73   11  494 8491   17    2
  106   11   64  119    6   69   10  172    3  448   11 4309   29  692
    1 3387   11   14   68 1332    9    6    2  941  179    3    6   73
    2   18   14    1   11  604   49   11  127  216    2  198    3   28
 1496   98    2  116   11  196   14  575   11  455  703    1   27   14
   37 7232    6   35  176  281  154   55   11   89  216    2   18   24
    9 3418   11   58  134  386    2  106 6328    2    1   14  494  873
    3   11   58  134    1    2  450 1423]
b"I absolutely LOVED this movie when I was a kid. I cried every time I watched it. It wasn't weird to me. I totally identified with the characters. I would love to see it again (and hope I wont be disappointed!). Pufnstuf rocks!!!! I was really drawn in to the fantasy world. And to me the movie was loooong. I wonder if I ever saw the series and have confused them? The acting I thought wa

In [24]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size,embedding_dim,input_length = max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])
model.compile(optimizer = 'adam', loss = 'binary_crossentropy',metrics = ['accuracy'])
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 11526     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [27]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Train on 25000 samples, validate on 25000 samples
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10
25000/25000 [==============================] - 5s 210us/sample - loss: 0.4778 - acc: 0.7622 - val_loss: 0.3952 - val_acc: 0.8207
Epoch 2/10
25000/25000 [==============================] - 5s 203us/sample - loss: 0.2399 - acc: 0.9075 - val_loss: 0.4263 - val_acc: 0.8118
Epoch 3/10
25000/25000 [==============================] - 5s 206us/sample - loss: 0.1015 - acc: 0.9722 - val_loss: 0.5121 - val_acc: 0.8070
Epoch 4/10
25000/25000 [==============================] - 5s 188us/sample - loss: 0.0265 - acc: 0.9964 - val_loss: 0.6553 - val_acc: 0.7933
Epoch 5/10
25000/25000 [==============================] - 5s 185us/sample - loss: 0.0066 - acc: 0.9994 - val_loss: 0.6943 - val_acc: 0.8064
Epoch 6/10
25000/25000 [==============================] - 5s 185us/sample - loss: 0.0021 - acc: 1.0000 - val_loss: 0.7566 - val_acc: 0.8078
Epoch 7/10
25000/25000 [==============================] - 5s 185us/sample - loss: 9.3490e-04 - acc: 1.0000 - val_loss: 0.8113 - val_acc: 0.8078
Epoch 8/10
25000

In [ ]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)